# Taller dos- Procesamiento de lenguaje natural

Se entregan dos potenciales bases de datos:
1. más de 1.000 tweets relacionados con el tema energético nacional, divididos en 2 archivos, clasificados por empleados de una empresa energética como positivos y negativos. Los tweets están cortados.
2. 800 tweets de propósito general (archivo tweets_800.xlsx) clasificados en positivo (1), negativo (-1) y neutro (0)

Su tarea es:
- Utilizando Transformers, calificar su sentimiento y encontrar métricas de precisión, recall y F1 score.
- Obtener entidades nombradas (NER) y realizar un wordcloud de las mismas.
- Apoyado en word embeddings, realizar un gráfico t-sne para los conceptos claves del tema (explicar grupos potenciales que se observan en el gráfico).


In [2]:
import pandas as pd

In [4]:
df = pd.read_excel('tweets_800.xlsx')

In [5]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   TWEET ORIGINAL  800 non-null    object
 1   POLARIDAD       800 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 12.6+ KB


,TWEET ORIGINAL,POLARIDAD
0,"@robertobrenes Bueno, no es tanto lo mayor com...",-1
1,Acabo de ver una chamaca con un piercing de mo...,-1
2,"""Mae mañana llego a las 10, cuidado te quedas ...",-1
3,@rociomoram29 que mal y por eso es que están j...,-1
4,"@KristelS18 Acá tengo pero ugh, la canela no m...",-1
